In [ ]:
import json
import subprocess
import sys
import os
import time
from typing import List, Dict, Any
from dataclasses import dataclass
from enum import Enum

class TaskStatus(Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    FAILED = "failed"

@dataclass
class Task:
    id: int
    description: str
    status: TaskStatus
    code_to_execute: str = ""
    result: str = ""
    error: str = ""

class PythonAutoGPT:
    def __init__(self, objective: str):
        self.objective = objective
        self.tasks: List[Task] = []
        self.task_counter = 0
        self.max_iterations = 10
        self.current_iteration = 0

    def generate_tasks(self) -> List[str]:
        """Generate initial tasks based on the objective"""
        # This is a simplified task generation - in a real AutoGPT,
        # this would use an LLM to break down the objective.

        common_patterns = {
            "data analysis": [
                "Import necessary libraries (pandas, numpy, matplotlib)",
                "Load or generate sample data",
                "Perform basic data exploration",
                "Create visualizations",
                "Generate summary statistics"
            ],
            "web app": [
                "Set up Flask/FastAPI application structure",
                "Create basic routes and endpoints",
                "Add HTML templates",
                "Implement core functionality",
                "Test the application"
            ],
            "algorithm": [
                "Define the algorithm structure",
                "Implement core logic",
                "Create test cases",
                "Optimize performance",
                "Document the solution"
            ],
            "file processing": [
                "Set up file handling utilities",
                "Read and parse input files",
                "Process data according to requirements",
                "Generate output files",
                "Validate results"
            ]
        }

        # Simple keyword matching to determine task type.
        objective_lower = self.objective.lower()

        if any(word in objective_lower for word in ["data", "analysis", "csv", "excel"]):
            return common_patterns["data analysis"]
        elif any(word in objective_lower for word in ["web", "app", "server", "api"]):
            return common_patterns["web app"]
        elif any(word in objective_lower for word in ["algorithm", "sort", "search", "optimize"]):
            return common_patterns["algorithm"]
        elif any(word in objective_lower for word in ["file", "process", "convert", "parse"]):
            return common_patterns["file processing"]
        else:
            # Default generic tasks.
            return [
                "Analyze the objective and requirements",
                "Design the solution approach",
                "Implement core functionality",
                "Test and validate the solution",
                "Refine and optimize"
            ]

    def create_task(self, description: str) -> Task:
        """Create a new task"""
        self.task_counter += 1
        task = Task(
            id=self.task_counter,
            description=description,
            status=TaskStatus.PENDING
        )
        self.tasks.append(task)
        return task

    def generate_code_for_task(self, task: Task) -> str:
        """Generate Python code for a given task"""
        # This is a simplified code generation - in a real AutoGPT,
        # this would use an LLM to generate appropriate code.

        task_desc = task.description.lower()

        if "import" in task_desc and "libraries" in task_desc:
            return """
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['savefig.bbox'] = 'tight'

print("Libraries imported successfully!")
print("Matplotlib backend set to 'Agg' for file output")
"""
        elif "load" in task_desc and "data" in task_desc:
            return """
# Generate sample data for demonstration
import pandas as pd
import numpy as np
import os

# Check if sample_data folder exists and try to load real data
if os.path.exists('sample_data'):
    print("Found sample_data folder, attempting to load datasets...")
    data_files = []
    for file in os.listdir('sample_data'):
        if file.endswith('.csv'):
            data_files.append(file)

    if data_files:
        print(f"Found CSV files: {data_files}")
        # Load the first CSV file found
        file_path = os.path.join('sample_data', data_files[0])
        print(f"Loading: {file_path}")
        df = pd.read_csv(file_path)
        print(f"Real data loaded with shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        print(df.head())
    else:
        print("No CSV files found in sample_data folder")
        # Fallback to generated data
        np.random.seed(42)
        data = {
            'date': pd.date_range('2024-01-01', periods=100),
            'value': np.random.randn(100).cumsum(),
            'category': np.random.choice(['A', 'B', 'C'], 100)
        }
        df = pd.DataFrame(data)
        print(f"Generated sample data with shape: {df.shape}")
        print(df.head())
else:
    print("sample_data folder not found, generating sample data...")
    # Create sample dataset
    np.random.seed(42)
    data = {
        'date': pd.date_range('2024-01-01', periods=100),
        'sales': np.random.uniform(1000, 10000, 100),
        'temperature': np.random.normal(25, 10, 100),
        'category': np.random.choice(['Electronics', 'Clothing', 'Books', 'Home'], 100),
        'region': np.random.choice(['North', 'South', 'East', 'West'], 100),
        'price': np.random.uniform(10, 1000, 100),
        'score': np.random.normal(75, 15, 100)
    }
    df = pd.DataFrame(data)
    print(f"Sample data created with shape: {df.shape}")
    print(df.head())

# Ensure we have the dataframe for later use
print(f"\\nDataFrame 'df' is ready with {df.shape[0]} rows and {df.shape[1]} columns")
"""
        elif "exploration" in task_desc or "explore" in task_desc:
            return """
# Data exploration
print("\\n" + "="*50)
print("DATA EXPLORATION")
print("="*50)

print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\\nData types:")
print(df.dtypes)
print(f"\\nMissing values:")
print(df.isnull().sum())
print(f"\\nFirst few rows:")
print(df.head())
print(f"\\nLast few rows:")
print(df.tail())

# Basic statistics for numerical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    print(f"\\nNumerical columns: {list(numeric_cols)}")
    print("\\nBasic statistics:")
    print(df[numeric_cols].describe())

# Unique values for categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
if len(categorical_cols) > 0:
    print(f"\\nCategorical columns: {list(categorical_cols)}")
    for col in categorical_cols:
        unique_count = df[col].nunique()
        print(f"\\n{col}: {unique_count} unique values")
        if unique_count <= 10:  # Show values if not too many
            print(df[col].value_counts())
        else:
            print(f"Top 5 most frequent values:")
            print(df[col].value_counts().head())
"""
        elif "visualization" in task_desc:
            return """
# Create visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import os

print("\\n" + "="*50)
print("CREATING VISUALIZATIONS")
print("="*50)

# Ensure we're using the Agg backend
import matplotlib
matplotlib.use('Agg')

# Create output directory for plots
os.makedirs('plots', exist_ok=True)

# Set style for better looking plots
plt.style.use('default')
sns.set_palette("husl")

# Get numeric and categorical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Filter out datetime columns from numeric columns
datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in datetime_cols]

print(f"Creating visualizations for {len(numeric_cols)} numeric columns and {len(categorical_cols)} categorical columns")

# 1. DISTRIBUTION PLOTS FOR NUMERIC COLUMNS
if len(numeric_cols) > 0:
    print("\\n1. Creating distribution plots for numeric columns...")

    # Calculate subplot layout
    n_numeric = min(4, len(numeric_cols))  # Limit to 4 plots
    if n_numeric == 1:
        fig, axes = plt.subplots(1, 1, figsize=(8, 6))
        axes = [axes]
    elif n_numeric == 2:
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    else:
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()

    for i, col in enumerate(numeric_cols[:n_numeric]):
        try:
            # Create histogram
            data_clean = df[col].dropna()
            axes[i].hist(data_clean, bins=30, alpha=0.7, edgecolor='black', color='skyblue')
            axes[i].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Frequency')
            axes[i].grid(True, alpha=0.3)

            # Add statistics text
            mean_val = data_clean.mean()
            median_val = data_clean.median()
            axes[i].axvline(mean_val, color='red', linestyle='--', alpha=0.7, label=f'Mean: {mean_val:.2f}')
            axes[i].axvline(median_val, color='green', linestyle='--', alpha=0.7, label=f'Median: {median_val:.2f}')
            axes[i].legend()

        except Exception as e:
            print(f"Error plotting {col}: {e}")

    # Hide unused subplots
    for i in range(n_numeric, len(axes)):
        axes[i].set_visible(False)

    plt.tight_layout()
    plt.savefig('plots/numeric_distributions.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print("   ✅ Numeric distributions saved as 'plots/numeric_distributions.png'")

# 2. CATEGORICAL PLOTS
if len(categorical_cols) > 0:
    print("\\n2. Creating categorical distribution plots...")

    n_categorical = min(2, len(categorical_cols))
    if n_categorical == 1:
        fig, axes = plt.subplots(1, 1, figsize=(10, 6))
        axes = [axes]
    else:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    for i, col in enumerate(categorical_cols[:n_categorical]):
        try:
            value_counts = df[col].value_counts().head(10)  # Top 10 categories
            colors = plt.cm.Set3(np.linspace(0, 1, len(value_counts)))

            bars = axes[i].bar(range(len(value_counts)), value_counts.values, color=colors)
            axes[i].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Count')
            axes[i].set_xticks(range(len(value_counts)))
            axes[i].set_xticklabels(value_counts.index, rotation=45, ha='right')
            axes[i].grid(True, alpha=0.3, axis='y')

            # Add value labels on bars
            for bar, count in zip(bars, value_counts.values):
                height = bar.get_height()
                axes[i].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                           f'{int(count)}', ha='center', va='bottom', fontsize=9)

        except Exception as e:
            print(f"Error plotting {col}: {e}")

    # Hide unused subplot if only one categorical column
    if n_categorical == 1 and len(axes) > 1:
        axes[1].set_visible(False)

    plt.tight_layout()
    plt.savefig('plots/categorical_distributions.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print("   ✅ Categorical distributions saved as 'plots/categorical_distributions.png'")

# 3. CORRELATION HEATMAP
if len(numeric_cols) > 1:
    print("\\n3. Creating correlation heatmap...")

    fig, ax = plt.subplots(figsize=(10, 8))

    # Calculate correlation matrix
    corr_matrix = df[numeric_cols].corr()

    # Create heatmap
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Hide upper triangle
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f')

    plt.title('Correlation Matrix of Numeric Variables', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig('plots/correlation_heatmap.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print("   ✅ Correlation heatmap saved as 'plots/correlation_heatmap.png'")

# 4. SCATTER PLOTS
if len(numeric_cols) >= 2:
    print("\\n4. Creating scatter plots...")

    # Create scatter plot for first two numeric columns
    fig, ax = plt.subplots(figsize=(10, 8))

    x_col = numeric_cols[0]
    y_col = numeric_cols[1]

    # Create scatter plot with color coding if we have categorical data
    if len(categorical_cols) > 0:
        # Color by first categorical column
        color_col = categorical_cols[0]
        unique_categories = df[color_col].unique()[:10]  # Limit to 10 categories
        colors = plt.cm.tab10(np.linspace(0, 1, len(unique_categories)))

        for i, category in enumerate(unique_categories):
            mask = df[color_col] == category
            plt.scatter(df[mask][x_col], df[mask][y_col],
                       alpha=0.6, s=50, color=colors[i], label=str(category))

        plt.legend(title=color_col, bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        plt.scatter(df[x_col], df[y_col], alpha=0.6, s=50, color='coral')

    plt.xlabel(x_col, fontsize=12)
    plt.ylabel(y_col, fontsize=12)
    plt.title(f'Scatter Plot: {x_col} vs {y_col}', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('plots/scatter_plot.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print("   ✅ Scatter plot saved as 'plots/scatter_plot.png'")

# 5. BOX PLOTS (if we have both numeric and categorical data)
if len(numeric_cols) > 0 and len(categorical_cols) > 0:
    print("\\n5. Creating box plots...")

    fig, ax = plt.subplots(figsize=(12, 6))

    # Use first numeric and first categorical column
    numeric_col = numeric_cols[0]
    categorical_col = categorical_cols[0]

    # Create box plot
    df.boxplot(column=numeric_col, by=categorical_col, ax=ax, figsize=(12, 6))
    plt.suptitle('')  # Remove the automatic title
    plt.title(f'Box Plot: {numeric_col} by {categorical_col}', fontsize=14, fontweight='bold')
    plt.xlabel(categorical_col, fontsize=12)
    plt.ylabel(numeric_col, fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('plots/box_plots.png', dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()
    print("   ✅ Box plots saved as 'plots/box_plots.png'")

# 6. SUMMARY DASHBOARD
print("\\n6. Creating summary dashboard...")

fig = plt.figure(figsize=(16, 12))

# Create a 3x2 grid
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Dataset overview (text)
ax1 = fig.add_subplot(gs[0, 0])
ax1.axis('off')
overview_text = f'''Dataset Overview:
• Shape: {df.shape[0]} rows × {df.shape[1]} columns
• Numeric columns: {len(numeric_cols)}
• Categorical columns: {len(categorical_cols)}
• Missing values: {df.isnull().sum().sum()}
• Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB'''

ax1.text(0.05, 0.95, overview_text, transform=ax1.transAxes,
         fontsize=11, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))

# Plot 2: First numeric column histogram
if len(numeric_cols) > 0:
    ax2 = fig.add_subplot(gs[0, 1])
    col = numeric_cols[0]
    ax2.hist(df[col].dropna(), bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    ax2.set_title(f'Distribution: {col}', fontweight='bold')
    ax2.grid(True, alpha=0.3)

# Plot 3: First categorical column
if len(categorical_cols) > 0:
    ax3 = fig.add_subplot(gs[1, 0])
    col = categorical_cols[0]
    value_counts = df[col].value_counts().head(8)
    ax3.bar(range(len(value_counts)), value_counts.values, color='lightcoral')
    ax3.set_title(f'Top Categories: {col}', fontweight='bold')
    ax3.set_xticks(range(len(value_counts)))
    ax3.set_xticklabels(value_counts.index, rotation=45, ha='right')
    ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Correlation heatmap (small version)
if len(numeric_cols) > 1:
    ax4 = fig.add_subplot(gs[1, 1])
    corr_matrix = df[numeric_cols].corr()
    im = ax4.imshow(corr_matrix, cmap='RdYlBu_r', aspect='auto', vmin=-1, vmax=1)
    ax4.set_title('Correlation Matrix', fontweight='bold')
    ax4.set_xticks(range(len(corr_matrix.columns)))
    ax4.set_yticks(range(len(corr_matrix.columns)))
    ax4.set_xticklabels(corr_matrix.columns, rotation=45, ha='right')
    ax4.set_yticklabels(corr_matrix.columns)

# Plot 5: Scatter plot
if len(numeric_cols) >= 2:
    ax5 = fig.add_subplot(gs[2, :])
    x_col, y_col = numeric_cols[0], numeric_cols[1]
    ax5.scatter(df[x_col], df[y_col], alpha=0.6, s=30, color='green')
    ax5.set_xlabel(x_col)
    ax5.set_ylabel(y_col)
    ax5.set_title(f'Scatter: {x_col} vs {y_col}', fontweight='bold')
    ax5.grid(True, alpha=0.3)

plt.suptitle('Data Analysis Dashboard', fontsize=16, fontweight='bold')
plt.savefig('plots/dashboard.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.close()
print("   ✅ Dashboard saved as 'plots/dashboard.png'")

# List all created files
print("\\n" + "="*50)
print("📊 VISUALIZATION SUMMARY")
print("="*50)

plot_files = []
plots_dir = 'plots'
if os.path.exists(plots_dir):
    for file in os.listdir(plots_dir):
        if file.endswith('.png'):
            file_path = os.path.join(plots_dir, file)
            file_size = os.path.getsize(file_path) / 1024  # Size in KB
            plot_files.append((file, file_size))

if plot_files:
    print("Generated visualization files:")
    for filename, size in plot_files:
        print(f"  ✅ {filename} ({size:.1f} KB)")
else:
    print("  ❌ No plot files found")

print(f"\\nTotal plots created: {len(plot_files)}")
print("All visualizations are saved in the 'plots/' directory")
"""
        elif "summary" in task_desc and "statistics" in task_desc:
            return """
# Generate comprehensive summary statistics
print("\\n" + "="*60)
print("COMPREHENSIVE SUMMARY STATISTICS")
print("="*60)

# Dataset overview
print(f"Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Numerical summary
numeric_cols = df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    print(f"\\n📊 NUMERICAL VARIABLES ({len(numeric_cols)} columns)")
    print("-" * 50)
    print(df[numeric_cols].describe())

    # Additional statistics
    print("\\nAdditional Statistics:")
    for col in numeric_cols:
        print(f"\\n{col}:")
        print(f"  • Range: {df[col].min():.3f} to {df[col].max():.3f}")
        print(f"  • IQR: {df[col].quantile(0.75) - df[col].quantile(0.25):.3f}")
        try:
            print(f"  • Skewness: {df[col].skew():.3f}")
        except:
            print(f"  • Skewness: Unable to calculate")
        print(f"  • Missing values: {df[col].isnull().sum()} ({df[col].isnull().sum()/len(df)*100:.1f}%)")

# Categorical summary
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
if len(categorical_cols) > 0:
    print(f"\\n📋 CATEGORICAL VARIABLES ({len(categorical_cols)} columns)")
    print("-" * 50)
    for col in categorical_cols:
        unique_count = df[col].nunique()
        mode_value = df[col].mode().iloc[0] if len(df[col].mode()) > 0 else "N/A"
        print(f"\\n{col}:")
        print(f"  • Unique values: {unique_count}")
        print(f"  • Most frequent: '{mode_value}' ({df[col].value_counts().iloc[0]} times)")
        print(f"  • Missing values: {df[col].isnull().sum()} ({df[col].isnull().sum()/len(df)*100:.1f}%)")

        if unique_count <= 10:
            print(f"  • Value distribution:")
            for value, count in df[col].value_counts().items():
                percentage = count / len(df) * 100
                print(f"    - '{value}': {count} ({percentage:.1f}%)")

# Correlation analysis for numerical variables
if len(numeric_cols) > 1:
    print(f"\\n🔗 CORRELATION ANALYSIS")
    print("-" * 50)
    corr_matrix = df[numeric_cols].corr()

    # Find highly correlated pairs
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_value = corr_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:  # Threshold for "high" correlation
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_value))

    if high_corr_pairs:
        print("High correlations (|r| > 0.5):")
        for col1, col2, corr in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
            print(f"  • {col1} ↔ {col2}: r = {corr:.3f}")
    else:
        print("No strong correlations found (all |r| ≤ 0.5)")

# Data quality assessment
print(f"\\n🔍 DATA QUALITY ASSESSMENT")
print("-" * 50)
total_missing = df.isnull().sum().sum()
print(f"Total missing values: {total_missing} ({total_missing/(df.shape[0]*df.shape[1])*100:.2f}% of all values)")

if total_missing > 0:
    print("\\nColumns with missing values:")
    missing_cols = df.isnull().sum()
    missing_cols = missing_cols[missing_cols > 0].sort_values(ascending=False)
    for col, count in missing_cols.items():
        print(f"  • {col}: {count} missing ({count/len(df)*100:.1f}%)")

# Duplicate rows
duplicate_count = df.duplicated().sum()
print(f"\\nDuplicate rows: {duplicate_count} ({duplicate_count/len(df)*100:.2f}%)")

# Generate insights
print(f"\\n💡 KEY INSIGHTS")
print("-" * 50)
insights = []

# Check for highly correlated variables
if len(high_corr_pairs) > 0:
    insights.append(f"Found {len(high_corr_pairs)} highly correlated variable pairs")

# Check for missing data
if total_missing > 0:
    insights.append(f"Dataset has {total_missing} missing values across {len(missing_cols)} columns")

# Check for duplicates
if duplicate_count > 0:
    insights.append(f"Dataset contains {duplicate_count} duplicate rows")

# Check data distribution
if len(numeric_cols) > 0:
    highly_skewed = []
    for col in numeric_cols:
        try:
            skewness = abs(df[col].skew())
            if skewness > 2:
                highly_skewed.append(col)
        except:
            pass
    if highly_skewed:
        insights.append(f"Highly skewed variables detected: {', '.join(highly_skewed)}")

if insights:
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")
else:
    print("• Dataset appears to be well-structured with no major quality issues")

print("\\n" + "="*60)
print("ANALYSIS COMPLETE! 🎉")
print("="*60)
"""
        else:
            # Generic task code
            return f"""
# Task: {task.description}
print("Executing task: {task.description}")
print("Task implementation would go here...")
print("Task completed successfully!")
"""

    def execute_code(self, code: str) -> tuple[str, str]:
        """Execute Python code and return output and error"""
        try:
            # Build cumulative code by combining all previous successful task codes.
            cumulative_code = ""

            # Add all previously executed successful task codes.
            for task in self.tasks:
                if task.status == TaskStatus.COMPLETED and task.code_to_execute:
                    cumulative_code += task.code_to_execute + "\n\n"

            # Add current code.
            full_code = cumulative_code + code

            # Create a temporary file to execute the code.
            with open('temp_code.py', 'w') as f:
                f.write(full_code)

            # Execute the code and capture output.
            result = subprocess.run(
                [sys.executable, 'temp_code.py'],
                capture_output=True,
                text=True,
                timeout=30
            )

            # Clean up
            if os.path.exists('temp_code.py'):
                os.remove('temp_code.py')

            return result.stdout, result.stderr

        except subprocess.TimeoutExpired:
            return "", "Code execution timed out"
        except Exception as e:
            return "", f"Execution error: {str(e)}"

    def execute_task(self, task: Task) -> bool:
        """Execute a single task"""
        print(f"\\n--- Executing Task {task.id}: {task.description} ---")

        task.status = TaskStatus.IN_PROGRESS

        # Generate code for the task.
        task.code_to_execute = self.generate_code_for_task(task)

        print("Generated code:")
        print("```python")
        print(task.code_to_execute)
        print("```")

        # Execute the code.
        output, error = self.execute_code(task.code_to_execute)

        task.result = output
        task.error = error

        if error:
            print(f"❌ Task failed with error: {error}")
            task.status = TaskStatus.FAILED
            return False
        else:
            print(f"✅ Task completed successfully!")
            if output:
                print("Output:")
                print(output)
            task.status = TaskStatus.COMPLETED
            return True

    def should_continue(self) -> bool:
        """Determine if the agent should continue with more tasks"""
        if self.current_iteration >= self.max_iterations:
            print(f"\\n🛑 Reached maximum iterations ({self.max_iterations})")
            return False

        pending_tasks = [t for t in self.tasks if t.status == TaskStatus.PENDING]
        if not pending_tasks:
            print("\\n✅ All tasks completed!")
            return False

        failed_tasks = [t for t in self.tasks if t.status == TaskStatus.FAILED]
        if len(failed_tasks) > len(self.tasks) * 0.5:  # If more than 50% failed
            print("\\n❌ Too many tasks failed, stopping execution")
            return False

        return True

    def run(self):
        """Main execution loop"""
        print(f"🚀 Starting AutoGPT Agent")
        print(f"Objective: {self.objective}")
        print("=" * 60)

        # Generate initial tasks.
        task_descriptions = self.generate_tasks()
        for desc in task_descriptions:
            self.create_task(desc)

        print(f"\\n📋 Generated {len(self.tasks)} tasks:")
        for task in self.tasks:
            print(f"  {task.id}. {task.description}")

        # Execute tasks.
        while self.should_continue():
            self.current_iteration += 1
            print(f"\\n🔄 Iteration {self.current_iteration}")

            # Find next pending task.
            pending_tasks = [t for t in self.tasks if t.status == TaskStatus.PENDING]
            if not pending_tasks:
                break

            current_task = pending_tasks[0]
            success = self.execute_task(current_task)

            # Brief pause between tasks.
            time.sleep(1)

        # Final summary.
        self.print_summary()

    def print_summary(self):
        """Print execution summary"""
        print("\\n" + "=" * 60)
        print("📊 EXECUTION SUMMARY")
        print("=" * 60)

        completed = len([t for t in self.tasks if t.status == TaskStatus.COMPLETED])
        failed = len([t for t in self.tasks if t.status == TaskStatus.FAILED])
        pending = len([t for t in self.tasks if t.status == TaskStatus.PENDING])

        print(f"Objective: {self.objective}")
        print(f"Total Tasks: {len(self.tasks)}")
        print(f"✅ Completed: {completed}")
        print(f"❌ Failed: {failed}")
        print(f"⏳ Pending: {pending}")
        print(f"🔄 Iterations: {self.current_iteration}")

        print("\\nTask Details:")
        for task in self.tasks:
            status_emoji = {
                TaskStatus.COMPLETED: "✅",
                TaskStatus.FAILED: "❌",
                TaskStatus.PENDING: "⏳",
                TaskStatus.IN_PROGRESS: "🔄"
            }
            print(f"  {status_emoji[task.status]} {task.id}. {task.description}")

        # Check if plots were created
        if os.path.exists('plots') and any(f.endswith('.png') for f in os.listdir('plots')):
            print("\\n🎨 Generated Visualizations:")
            for file in os.listdir('plots'):
                if file.endswith('.png'):
                    print(f"  📊 plots/{file}")


# Example usage.
if __name__ == "__main__":
    # Example objectives you can try:
    objectives = [
        "Perform data analysis on a sample dataset with visualizations",
        "Create a simple file processing algorithm",
        "Build a basic web application structure",
        "Implement a sorting algorithm with tests"
    ]

    print("Available example objectives:")
    for i, obj in enumerate(objectives, 1):
        print(f"{i}. {obj}")

    # Use the first objective as default.
    selected_objective = objectives[0]
    print(f"\\nRunning with objective: {selected_objective}")

    # Create and run the agent.
    agent = PythonAutoGPT(selected_objective)
    agent.run()
